# Day 12: Hot Springs

In [ ]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val springs = Path("input.txt").readLines()
    .map {
        val (line, operational) = it.split(' ')
        line to operational.split(',').map { it.toInt() }
    }

I originally used the brute force method for part 1, computing and returning every possible combination in a simple function. For part 2, I optimized that using sequences, which made it space efficient but not yet time efficient. My thanks go to Roman Elizarov and his AOC Diary, in which he mentioned memoization. After reading up on memoization and a somewhat sleepless night, here's the final algorithm, computing the sum directly and using memoization to reuse intermediate results.

In [ ]:
class Counter(val brokenCount: Int, val operationals: Array<Int>, val line: String) {
    val memo = Array(line.length + 1) {
        Array(brokenCount + 1) {
            LongArray(operationals.sum() + 1) { -1 }
        }
    }
    
    fun combinations(index: Int, i1: Int, i2: Int): Long {
        if (memo[index][i1][i2] != -1L) {
            return memo[index][i1][i2]
        }
        
        if (i1 >= brokenCount && i2 >= operationals.size) {
            memo[index][i1][i2] = 1
            return 1
        }
        
        var sum = 0L
        if (i1 < brokenCount && (line[index] == '?' || line[index] == '.')) {
            sum += combinations(index + 1, i1 + 1, i2)
        }
        if (i2 < operationals.size) {
            var skip = false
            for (i in 0 until operationals[i2]) {
                val c = if (i2 > 0 && i == 0) '.' else '#'
                if (line[index + i] != '?' && c != line[index + i]) {
                    skip = true
                }
            }
            if (!skip) {
                sum += combinations(index + operationals[i2], i1, i2 + 1)
            }
        }
        memo[index][i1][i2] = sum
        return sum;
    }
}

fun findArrangements(line: String, operational: List<Int>): Long {
    val brokenCount = line.length - operational.sum() - (operational.size - 1)
    
    return Counter(brokenCount,
        operational.mapIndexed { i, o -> if (i == 0) o else o + 1 }.toTypedArray(),
        line
    ).combinations(0, 0, 0)
}

## Part 1

In [ ]:
springs.sumOf { (line, operational) -> findArrangements(line, operational) }

## Part 2

In [ ]:
springs
    .map { (line, operational) -> List(5) {line}.joinToString("?") to List(5) {operational}.flatten() }
    .sumOf { (line, operational) -> findArrangements(line, operational) }